In [1]:
from utilidades import *
import warnings
warnings.filterwarnings('ignore')

# Deep Clustering

O Deep Learning envolve a construção e treinamento de redes neurais artificiais profundas para realizar tarefas complexas de aprendizado, como reconhecimento de padrões, classificação e clusterização. Nos últimos anos, o uso de técnicas de Deep Learning revolucionou a forma como a clusterização é realizada, permitindo uma abordagem mais flexível e precisa na identificação de padrões complexos nos dados. A principal vantagem do Deep Learning em relação a abordagens tradicionais de clusterização está na sua capacidade de aprender automaticamente representações de características hierárquicas dos dados, sem a necessidade de uma definição explícita de características. Isso permite que modelos de Deep Learning identifiquem padrões abstratos e latentes nos dados, que podem ser cruciais para a formação de clusters significativos.

**<h1>Deep Clustering Network (DCN)</h1>**

Deep Clustering Network (DCN) é uma abordagem avançada que combina técnicas de Deep Learning com a clusterização tradicional para realizar a tarefa de clusterização de dados de forma mais eficaz. O objetivo do DCN é aprender representações de dados de alta qualidade, que são então utilizadas para realizar a clusterização. Ao contrário de abordagens tradicionais, que muitas vezes requerem a definição manual de características (as features), o DCN permite que o próprio modelo aprenda representações relevantes durante o processo de treinamento.

A ideia central por trás do DCN é treinar uma rede neural profunda (geralmente uma autoencoder) para codificar os dados de entrada em um espaço latente de dimensão reduzida. Esse espaço latente captura as informações mais importantes dos dados e é projetado de tal forma que os pontos de dados similares são mapeados para regiões próximas no espaço latente. Uma vez que as representações latentes são aprendidas, técnicas de clusterização tradicionais, como o K-means, são aplicadas para agrupar os pontos de dados no espaço latente em clusters.


# Data

In [2]:
X_iris, y_iris, iris_target_names = get_iris_data()
X_wine, y_wine, wine_target_names = get_wine_data()
X_syn_ctrl, y_syn_ctrl, syn_ctrl_target_names = get_synthetic_control_data()

# Clusterização

In [57]:
# Importando os pacotes necessários
import tensorflow as tf
from scipy.stats import zscore
from sklearn.cluster import KMeans

# Normalizando os dados
X_syn_ctrl = zscore(X_syn_ctrl)
X_iris = zscore(X_iris)
X_wine = zscore(X_wine)

In [58]:
def return_autoencoder(input_dim, encoding_dim):

    input_layer = tf.keras.layers.Input(shape=(input_dim,))
    encoder = tf.keras.layers.Dense(encoding_dim, activation='sigmoid')(input_layer)
    decoder = tf.keras.layers.Dense(input_dim, activation='softmax')(encoder)

    autoencoder = tf.keras.models.Model(inputs=input_layer, outputs=decoder)

    return autoencoder

In [59]:
# Definindo o autoencoder 
iris_input_dim = X_iris.shape[1]
wine_input_dim = X_wine.shape[1]
syn_ctrl_input_dim = X_syn_ctrl.shape[1]

encoding_dim = 2

autoencoder_syn_ctrl = return_autoencoder(syn_ctrl_input_dim, encoding_dim)
autoencoder_wine = return_autoencoder(wine_input_dim, encoding_dim)
autoencoder_iris = return_autoencoder(iris_input_dim, encoding_dim)

In [60]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_syn_ctrl, y_syn_ctrl, test_size=0.3)

In [61]:
# Compilando e treinando os autoencoders

# Synthetic Control
autoencoder_syn_ctrl.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_syn_ctrl.fit(X_train, X_train, epochs=25, batch_size=32, shuffle=True, validation_data=(X_test, X_test))

# Iris
autoencoder_iris.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_iris.fit(X_iris, X_iris, epochs=25, batch_size=8, shuffle=True)

# Wine
autoencoder_wine.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder_wine.fit(X_wine, X_wine, epochs=25, batch_size=8, shuffle=True)


Epoch 1/25


14/14 [==============================] - 1s 10ms/step - loss: 0.6779 - val_loss: 0.6686
Epoch 2/25
14/14 [==============================] - 0s 4ms/step - loss: 0.6613 - val_loss: 0.6548
Epoch 3/25
14/14 [==============================] - 0s 5ms/step - loss: 0.6461 - val_loss: 0.6414
Epoch 4/25
14/14 [==============================] - 0s 4ms/step - loss: 0.6316 - val_loss: 0.6282
Epoch 5/25
14/14 [==============================] - 0s 4ms/step - loss: 0.6170 - val_loss: 0.6153
Epoch 6/25
14/14 [==============================] - 0s 4ms/step - loss: 0.6027 - val_loss: 0.6020
Epoch 7/25
14/14 [==============================] - 0s 4ms/step - loss: 0.5876 - val_loss: 0.5872
Epoch 8/25
14/14 [==============================] - 0s 4ms/step - loss: 0.5701 - val_loss: 0.5684
Epoch 9/25
14/14 [==============================] - 0s 4ms/step - loss: 0.5480 - val_loss: 0.5462
Epoch 10/25
14/14 [==============================] - 0s 4ms/step - loss: 0.5245 - val_loss: 0.5239
Epoch 11/25
14/14 [==========

In [62]:
# Extract latent representations
encoder_model = tf.keras.models.Model(inputs=input_layer, outputs=encoder)
latent_representations = encoder_model.predict(x_test)

(150, 4)

In [65]:
autoencoder_syn_ctrl.layers[2]